# Fingerprint frequency estimation module

In [ ]:
import numpy as np
import cv2 as cv
from ipywidgets import interact
import pyfing as pf
from pyfing.utils.ffe import load_ffe_dataset
from pyfing.frequencies import compute_ridge_period_MAPE
from pyfing.utils.notebook import show
from pyfing.utils.drawing import draw_orientations, draw_frequencies

In [ ]:
# Paths
PATH_FFE = '../../datasets/FFE/'

## 1. Basic examples

Example: fingerprint segmentation with GMFS, orientation estimation with GBFOE, and frequency estimation with XSFFE.

In [ ]:
fingerprint = cv.imread(PATH_FFE + 'Good/110.png', cv.IMREAD_GRAYSCALE)
gmfs = pf.Gmfs()
gbfoe = pf.Gbfoe()
xsffe = pf.Xsffe()
segmentation_mask = gmfs.run(fingerprint)
orientations, _ = gbfoe.run(fingerprint, segmentation_mask)
frequencies = xsffe.run(fingerprint, segmentation_mask, orientations)
show(fingerprint, segmentation_mask, 
     draw_orientations(fingerprint, orientations, segmentation_mask, 1, 8, (0,255,0)),
     draw_frequencies(fingerprint, frequencies, segmentation_mask))

Example: fingerprint segmentation with SUFS, orientation estimation with SNFOE, and frequency estimation with SNFFE.

In [ ]:
fingerprint = cv.imread(PATH_FFE + 'Bad/00.png', cv.IMREAD_GRAYSCALE)
sufs = pf.Sufs()
snfoe = pf.Snfoe()
snffe = pf.Snffe()
segmentation_mask = sufs.run(fingerprint)
orientations, _ = snfoe.run(fingerprint, segmentation_mask)
frequencies = snffe.run(fingerprint, segmentation_mask, orientations)
show(fingerprint, segmentation_mask, 
     draw_orientations(fingerprint, orientations, segmentation_mask, 1, 16, (0,255,0)),
     draw_frequencies(fingerprint, frequencies, segmentation_mask))

## 2. Comparison with the ground truth

In [ ]:
ffe_bad = load_ffe_dataset(PATH_FFE, 'Bad')

In [ ]:
@interact(i=(0,49))
def test_ffe_bad(i=0):
    f, m, o, gt_rp = ffe_bad[i]
    rp = snffe.run(f, m, o)
    mape = compute_ridge_period_MAPE(rp, gt_rp, m)
    show((f, f'{i}'), (draw_frequencies(f, rp, m), f'MAPE={mape:.2f}%'))